## Anomaly & Detrend
This code shows as simple way to:
- Read in a .nc file using xarray
- Generate montly anomalies from climatology of monthly means
- detrend data by grid cell

Will Wieder Feb. 2020

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import cartopy
import cartopy.crs as ccrs
import cf_units as cf
import esmlab

# some resources for plotting
import matplotlib.pyplot as plt
import scipy  # Try scipy to detrend
import segment as sg
from ctsm_py import utils
from scipy import signal

%matplotlib inline

In [ ]:
## User defined options
model = "CLM5_GSWP3"
variables = ["GPP", "NBP", "TWS"]
pattern = (
    "/glade/p/cgd/tss/people/oleson/CLM_LAND_ONLY_RELEASE/CLM5/clm50_r270_1deg_GSWP3V1_iso_newpopd_hist/"
    "lnd/proc/tseries/month_1/clm50_r270_1deg_GSWP3V1_iso_newpopd_hist.clm2.h0.{var}.185001-201412.nc"
)
files = [pattern.format(var=var) for var in variables]

# Read in the data
# utils.time_set_mid corrects dates, to get Jan of first year
ds = utils.time_set_mid(
    xr.open_mfdataset(files, combine="by_coords", decode_times=True), "time"
)  # combine needed for newer xarray versions

var = variables  # redundant, but we'll keep using this for now because 'var' is used more later on...
area = ds.area
landfrac = ds.landfrac
# print(ds.var)

In [ ]:
# Select the time slice to analize,  here we'll do the last 25 years
# Use esmlab to calculate climatology & anomalies
# First we'll do this for every grid w/ monthly data

years = 25
months = years * 12

ds2 = ds.isel(time=slice(-months, None))  # Select last N years of data
ds_climo = esmlab.core.climatology(ds2, freq="mon")  # Calculate climatology
ds_anom = esmlab.core.anomaly(
    ds2, clim_freq="mon", time_coord_name="time"
)  # not sure how to use slice_mon_clim_time
ds_anom = ds_anom.where(
    ds_anom.get(var[0]).max(dim="time")
)  # mask out regions with no GPP for all variables
print(ds_anom.dims)

In [ ]:
# now remove trends from each grid cell
# detrend data using signal.detrend
# use apply_ufunc
# This works for just for one variable


def dtrend(anom, dim):
    # note: apply always moves core dimensions to the end
    return xr.apply_ufunc(
        signal.detrend,
        anom,
        input_core_dims=[[dim]],
        output_core_dims=[[dim]],
        kwargs={"axis": -1},
    )


dt_anom = dtrend(ds_anom.GPP.load().fillna(0), "time")

In [ ]:
# quick look at a grid cell

ilat = 60
ilon = 240
plt.plot(
    ds2.time["time.month"], ds2["GPP"].sel(lat=ilat, lon=ilon, method="nearest"), "o"
)
plt.plot(
    ds_climo.time["time.month"],
    ds_climo["GPP"].sel(lat=ilat, lon=ilon, method="nearest"),
    "-",
    lw=3,
)
plt.plot(
    ds_anom.time["time.month"],
    ds_anom["GPP"].sel(lat=ilat, lon=ilon, method="nearest"),
    "o",
)
plt.plot(
    dt_anom.time["time.month"], dt_anom.sel(lat=ilat, lon=ilon, method="nearest"), "*"
)
plt.legend(labels=("raw", "climo", "anom", "dt_anom"), frameon=False)
plt.axhline(0, color="black", lw=1)
plt.ylabel("GPP (gC/m2/s)")

### Global means, anomalies, & detrended anomalies
#### * These are still monthly results * 

In [ ]:
# mask out area for gridcells with no GPP for weights?
landUp = area * landfrac * 1e6  # area in km2, not m2
wgt = landUp / landUp.sum()  # weighting for each grid cell
spd = 3600 * 24  # Convert  to daily fluxes (gC/m2/d), from gc/m2/s
spy = spd * 365

global_ts = esmlab.statistics.weighted_sum(ds2.GPP * landUp, dim=("lat", "lon")) * spd
global_ts_anom = (
    esmlab.statistics.weighted_sum(ds_anom.GPP * landUp, dim=("lat", "lon")) * spd
)
global_ts_dt = signal.detrend(global_ts_anom)

# Then makes some plots
plt.plot(global_ts.time, global_ts_anom, "-")
plt.plot(global_ts.time, global_ts_dt, "-")
plt.ylabel("Global monthly GPP anomalies (gC/m2/d)")
plt.legend(labels=("monthly anomalies", "detrended anomalies"))

### Convert monthly to annual

In [ ]:
# These values don't make any sense, let's have a closer look.

# apply iterates through all variables in a dataset and applies the function to each variable.
# ?? Would be nice to do this differently for fluxes & stocks ??
ann_mean = utils.weighted_annual_mean(ds2["GPP"]) * spy
ann_mean.attrs["units"] = "gC m-2 y-1"
print(ann_mean)
# print(ds2.apply_ufunc(utils.weighted_annual_mean))

# ann_mean_anom  = esmlab.core.anomaly(ann_mean, clim_freq='mon', time_coord_name='time') #not sure how to use slice_mon_clim_time

global_ann_ts = (
    esmlab.statistics.weighted_sum(ann_mean * landUp, dim=("lat", "lon")) * 1e-15
)  # PgC y-1
global_ann_dt = signal.detrend(global_ann_ts)
# global_ann_dt.attrs['units'] = 'PgC y-1'
print(global_ann_ts.values)
# plt.plot(global_ann_ts.time, global_ann_ts, '-' )
plt.plot(global_ann_ts.time, global_ann_dt, "-")
plt.ylabel("Global annual GPP anomalies (PgC/y)")